In [51]:
# 원화마켓o, 유의종목x, BTCx 코인에 대해서
# 실시간으로 1분봉의 정보들을 가져오고
# 1초마다 체크해서
# 1분안에 이전 1분봉대비 10%가 오른 코인에 대해서
# 원화의 20%를 시장가매수하고
# 1초마다 체크해서
# 5%이상의 수익이 나오면 시장가매도를 한다.
# 계속 반복한다.

In [52]:
import requests, time, jwt, uuid, pyupbit, datetime
import pandas as pd
import numpy as np
from sendMail import send_email
import json

f = open("업비트정보.private.json", "r")
api_key = json.load(f)
f.close()

A_key = api_key["accessKey"]  # 본인 access_key 키로 변경
S_key = api_key["secretKey"]  # 본인 secret_key 키로 변경

In [53]:
tickers = pyupbit.get_tickers(fiat="KRW") # 유의종목포함했음

## 몇 프로 이상 상승하면 수익화 할 것인지
ascent = 5  # 5% 상승
## 몇 프로 이상 하락하면 손절 할 것인지
downhill = 5  # 5% 하락

In [54]:
tickers

['KRW-BTC',
 'KRW-ETH',
 'KRW-NEO',
 'KRW-MTL',
 'KRW-XRP',
 'KRW-ETC',
 'KRW-OMG',
 'KRW-SNT',
 'KRW-WAVES',
 'KRW-XEM',
 'KRW-QTUM',
 'KRW-LSK',
 'KRW-STEEM',
 'KRW-XLM',
 'KRW-ARDR',
 'KRW-ARK',
 'KRW-STORJ',
 'KRW-GRS',
 'KRW-REP',
 'KRW-ADA',
 'KRW-SBD',
 'KRW-POWR',
 'KRW-BTG',
 'KRW-ICX',
 'KRW-EOS',
 'KRW-TRX',
 'KRW-SC',
 'KRW-ONT',
 'KRW-ZIL',
 'KRW-POLYX',
 'KRW-ZRX',
 'KRW-LOOM',
 'KRW-BCH',
 'KRW-BAT',
 'KRW-IOST',
 'KRW-RFR',
 'KRW-CVC',
 'KRW-IQ',
 'KRW-IOTA',
 'KRW-HIFI',
 'KRW-ONG',
 'KRW-GAS',
 'KRW-UPP',
 'KRW-ELF',
 'KRW-KNC',
 'KRW-BSV',
 'KRW-THETA',
 'KRW-QKC',
 'KRW-BTT',
 'KRW-MOC',
 'KRW-ENJ',
 'KRW-TFUEL',
 'KRW-MANA',
 'KRW-ANKR',
 'KRW-AERGO',
 'KRW-ATOM',
 'KRW-TT',
 'KRW-CRE',
 'KRW-MBL',
 'KRW-WAXP',
 'KRW-HBAR',
 'KRW-MED',
 'KRW-MLK',
 'KRW-STPT',
 'KRW-ORBS',
 'KRW-VET',
 'KRW-CHZ',
 'KRW-STMX',
 'KRW-DKA',
 'KRW-HIVE',
 'KRW-KAVA',
 'KRW-AHT',
 'KRW-LINK',
 'KRW-XTZ',
 'KRW-BORA',
 'KRW-JST',
 'KRW-CRO',
 'KRW-TON',
 'KRW-SXP',
 'KRW-HUNT',
 'KRW-PLA

In [55]:
while(1):
    for market_code in tickers:
        time.sleep(0.1)

        ## BTT는 너무 단위가 작아서..
        if market_code in ["KRW-BTC", "KRW-ETH", "KRW-XRP", "KRW-BTT"]:
            continue

        print(market_code, f'{pyupbit.get_current_price(market_code)}원')

        ## API로 업비트에서 내 계좌 조회
        my_exchange_account = pd.DataFrame(requests.get("https://api.upbit.com/v1/accounts",
                                                            headers={"Authorization": 'Bearer {}'.format(
                                                                jwt.encode({'access_key': A_key,
                                                                            'nonce': str(uuid.uuid4())}, S_key))}).json())
        now_krw = float(my_exchange_account[my_exchange_account['currency'] == 'KRW']['balance'].tail())

        ## 7000원이 없으면, 5분대기 후 재실행
        if now_krw < 7000:
            time.sleep(60 * 5)
            continue

        ## 1분전 대비 10%급등한지 검증
        ohlcv_value = pyupbit.get_ohlcv(market_code, interval="minute1")
        ohlcv_value['change'] = ohlcv_value['close'] - ohlcv_value['close'].shift(1)
        ohlcv_value['change_rate'] = (ohlcv_value['close'] / ohlcv_value['close'].shift(1)) * 100

        # 지금 코인을 구매한 상태인지 체크한다.
        was_coin_bought = False

        ## 10%이상 급등했을때
        if ohlcv_value['change_rate'][-1] > 110:
            # 시장가 구매
            if round(now_krw * 0.2) >= 7000:
                ## 주문 금액을 보유 원화의 20%로 지정
                order_amount = round(now_krw * 0.2)
            else:
                order_amount = 7000

            ## API로 업비트에서 시장가 매수 진행
            buy_market_order_data = pd.DataFrame.from_dict(pyupbit.Upbit(A_key, S_key)
                                                           .buy_market_order(market_code, order_amount), orient='index').T

            # 특정 주문 조회
            ## 조회할 주문 uuid
            order_uuid = buy_market_order_data.uuid[0]
            ## API로 업비트에서 특정 주문 조회
            specific_order = pd.DataFrame.from_dict(pyupbit.Upbit(A_key, S_key).get_order(order_uuid), orient='index').T
            buy_market_order_price = (round(float(specific_order['price'][0]) / float(specific_order['executed_volume'][0]), 2))
            send_email(specific_order['market'][0] + '을 매수가 ' + str(buy_market_order_price) + '원에 ' +
                  str(round(float(specific_order['executed_volume'][0]), 5)) + '개 구매',"급등코인매수!")
            print(specific_order['market'][0] + '을 매수가 ' + str(buy_market_order_price) + '원에 ' +
                  str(round(float(specific_order['executed_volume'][0]), 5)) + '개 구매')

            was_coin_bought = True
        else:
            print(f"{market_code}은 10%이상 급등하지 않았습니다. {ohlcv_value['change_rate'][-1]}%변동")

        ## 지금 매수한 상태면 계속 매도하기 위해서 대기한다.
        while(was_coin_bought):
            time.sleep(1)

            current_price = pd.DataFrame(requests.get("https://api.upbit.com/v1/ticker?markets=" + market_code,
                                                  headers={"Accept": "application/json"}).json())

            order_quantity = pyupbit.Upbit(A_key, S_key).get_balance(market_code)   # 특정 종목 보유 수량
            ## 현재구매한 코인의 시가가 구매가보다 ascent이상이면 수익화한다.
            ascent_rate = (current_price/buy_market_order_price)*100 - 100

            if(ascent_rate >= ascent):
                sell_market_order_data = pd.DataFrame.from_dict(
                            pyupbit.Upbit(A_key, S_key).sell_market_order(market_code, order_quantity), orient='index').T
                print(str(ascent) + '% 증가하여 시장가 매도 익절')
                send_mail(str(ascent) + '% 증가하여 시장가 매도 익절', "급등주 매도 익절!")

                was_coin_bought = False

            ## 현재구매한 코인의 시가가 구매가보다 downhill미만이면 손절한다
            downhill_rate = -ascent_rate

            if(downhill_rate > downhill):
                sell_market_order_data = pd.DataFrame.from_dict(
                            pyupbit.Upbit(A_key, S_key).sell_market_order(market_code, order_quantity), orient='index').T
                print(str(downhill) + '% 하락하여 시장가 매도 손절')
                send_mail(str(downhill) + '% 하락하여 시장가 매도 손절', "급등주 매도 손절ㅠ")

                was_coin_bought = False


KRW-NEO 11360원
KRW-NEO은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MTL 1350원
KRW-MTL은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ETC 28930원
KRW-ETC은 10%이상 급등하지 않았습니다. 99.96544574982723%변동
KRW-OMG 2045원
KRW-OMG은 10%이상 급등하지 않았습니다. 99.7560975609756%변동
KRW-SNT 37.3원
KRW-SNT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-WAVES 3660원
KRW-WAVES은 10%이상 급등하지 않았습니다. 100.13679890560876%변동
KRW-XEM 51.3원
KRW-XEM은 10%이상 급등하지 않았습니다. 99.80544747081711%변동
KRW-QTUM 3600원
KRW-QTUM은 10%이상 급등하지 않았습니다. 99.86130374479889%변동
KRW-LSK 1540원
KRW-LSK은 10%이상 급등하지 않았습니다. 100.32679738562092%변동
KRW-STEEM 273원
KRW-STEEM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-XLM 116원
KRW-XLM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ARDR 123원
KRW-ARDR은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ARK 454원
KRW-ARK은 10%이상 급등하지 않았습니다. 99.78021978021978%변동
KRW-STORJ 512원
KRW-STORJ은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-GRS 511원
KRW-GRS은 10%이상 급등하지 않았습니다. 99.8046875%변동
KRW-REP 12010원
KRW-REP은 10%이상 급등하지 않았습니다. 99.17491749174917%변동
KRW-ADA 495원
KRW-ADA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SBD 3445원
KRW-SBD은 10%이상 급등하지 않았

KRW-IQ 8.21원
KRW-IQ은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-IOTA 305원
KRW-IOTA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-HIFI 644원
KRW-HIFI은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ONG 393원
KRW-ONG은 10%이상 급등하지 않았습니다. 99.74619289340102%변동
KRW-GAS 3385원
KRW-GAS은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-UPP 80.6원
KRW-UPP은 10%이상 급등하지 않았습니다. 100.37359900373599%변동
KRW-ELF 245원
KRW-ELF은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-KNC 1095원
KRW-KNC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-BSV 56300원
KRW-BSV은 10%이상 급등하지 않았습니다. 100.53571428571428%변동
KRW-THETA 1425원
KRW-THETA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-QKC 14.1원
KRW-QKC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MOC 104원
KRW-MOC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ENJ 582원
KRW-ENJ은 10%이상 급등하지 않았습니다. 99.82847341337907%변동
KRW-TFUEL 74원
KRW-TFUEL은 10%이상 급등하지 않았습니다. 99.86522911051212%변동
KRW-MANA 935원
KRW-MANA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ANKR 35.2원
KRW-ANKR은 10%이상 급등하지 않았습니다. 99.43502824858759%변동
KRW-AERGO 184원
KRW-AERGO은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ATOM 18330원
KRW-ATOM은 10%이상 급등하지 않았습니다. 99.89100817438693%변동
KR

KRW-JST은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-CRO 104원
KRW-CRO은 10%이상 급등하지 않았습니다. 100.97087378640776%변동
KRW-TON 3000원
KRW-TON은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SXP 387원
KRW-SXP은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-HUNT 435원
KRW-HUNT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-PLA 308원
KRW-PLA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-DOT 8400원
KRW-DOT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SRM 639원
KRW-SRM은 10%이상 급등하지 않았습니다. 100.15673981191222%변동
KRW-MVL 5.48원
KRW-MVL은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-STRAX 746원
KRW-STRAX은 10%이상 급등하지 않았습니다. 99.8661311914324%변동
KRW-AQT 1655원
KRW-AQT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-GLM 324원
KRW-GLM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SSX 26.9원
KRW-SSX은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-META 42.8원
KRW-META은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-FCT2 74원
KRW-FCT2은 10%이상 급등하지 않았습니다. 99.86504723346829%변동
KRW-CBK 1000원
KRW-CBK은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SAND 930원
KRW-SAND은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-HUM 134원
KRW-HUM은 10%이상 급등하지 않았습니다. 99.25925925925925%변동
KRW-DOGE 116원
KRW-DOGE은 10%이상 급등하지 않았습니다. 99.1452991

KRW-CELO 970원
KRW-CELO은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-GMT 671원
KRW-GMT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-APT 19540원
KRW-APT은 10%이상 급등하지 않았습니다. 99.89775051124744%변동
KRW-SHIB 0.0184원
KRW-SHIB은 10%이상 급등하지 않았습니다. 99.45652173913044%변동
KRW-NEO 11320원
KRW-NEO은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MTL 1355원
KRW-MTL은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ETC 28960원
KRW-ETC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-OMG 2050원
KRW-OMG은 10%이상 급등하지 않았습니다. 99.7566909975669%변동
KRW-SNT 37.3원
KRW-SNT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-WAVES 3660원
KRW-WAVES은 10%이상 급등하지 않았습니다. 99.86357435197817%변동
KRW-XEM 51.1원
KRW-XEM은 10%이상 급등하지 않았습니다. 99.8046875%변동
KRW-QTUM 3600원
KRW-QTUM은 10%이상 급등하지 않았습니다. 99.7229916897507%변동
KRW-LSK 1590원
KRW-LSK은 10%이상 급등하지 않았습니다. 100.3154574132492%변동
KRW-STEEM 273원
KRW-STEEM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-XLM 116원
KRW-XLM은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ARDR 122원
KRW-ARDR은 10%이상 급등하지 않았습니다. 99.1869918699187%변동
KRW-ARK 453원
KRW-ARK은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-STORJ 513원
KRW-STORJ은 10%이상 급등하지 않았습니다. 100.0

KRW-IOST은 10%이상 급등하지 않았습니다. 100.73529411764706%변동
KRW-RFR 7.73원
KRW-RFR은 10%이상 급등하지 않았습니다. 100.12953367875647%변동
KRW-CVC 146원
KRW-CVC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-IQ 8.2원
KRW-IQ은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-IOTA 304원
KRW-IOTA은 10%이상 급등하지 않았습니다. 99.672131147541%변동
KRW-HIFI 645원
KRW-HIFI은 10%이상 급등하지 않았습니다. 100.78125%변동
KRW-ONG 392원
KRW-ONG은 10%이상 급등하지 않았습니다. 100.25575447570331%변동
KRW-GAS 3390원
KRW-GAS은 10%이상 급등하지 않았습니다. 99.85272459499264%변동
KRW-UPP 80원
KRW-UPP은 10%이상 급등하지 않았습니다. 99.7506234413965%변동
KRW-ELF 245원
KRW-ELF은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-KNC 1090원
KRW-KNC은 10%이상 급등하지 않았습니다. 99.54337899543378%변동
KRW-BSV 55990원
KRW-BSV은 10%이상 급등하지 않았습니다. 99.9642920906981%변동
KRW-THETA 1420원
KRW-THETA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-QKC 14.1원
KRW-QKC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MOC 103원
KRW-MOC은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-ENJ 582원
KRW-ENJ은 10%이상 급등하지 않았습니다. 99.82847341337907%변동
KRW-TFUEL 73.9원
KRW-TFUEL은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-MANA 933원
KRW-MANA은 10%이상 급등하지 않았습니다. 99.892933

KRW-AHT은 10%이상 급등하지 않았습니다. 99.88465974625144%변동
KRW-LINK 8835원
KRW-LINK은 10%이상 급등하지 않았습니다. 99.94343891402715%변동
KRW-XTZ 1495원
KRW-XTZ은 10%이상 급등하지 않았습니다. 99.66666666666667%변동
KRW-BORA 251원
KRW-BORA은 10%이상 급등하지 않았습니다. 99.60317460317461%변동
KRW-JST 35.3원
KRW-JST은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-CRO 102원
KRW-CRO은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-TON 2975원
KRW-TON은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-SXP 386원
KRW-SXP은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-HUNT 433원
KRW-HUNT은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-PLA 309원
KRW-PLA은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-DOT 8375원
KRW-DOT은 10%이상 급등하지 않았습니다. 100.05973715651135%변동
KRW-SRM 636원
KRW-SRM은 10%이상 급등하지 않았습니다. 99.84301412872841%변동
KRW-MVL 5.47원
KRW-MVL은 10%이상 급등하지 않았습니다. 99.81751824817518%변동
KRW-STRAX 746원
KRW-STRAX은 10%이상 급등하지 않았습니다. 100.0%변동
KRW-AQT 1660원
KRW-AQT은 10%이상 급등하지 않았습니다. 100.91185410334347%변동
KRW-GLM 323원
KRW-GLM은 10%이상 급등하지 않았습니다. 99.69135802469135%변동
KRW-SSX 27원
KRW-SSX은 10%이상 급등하지 않았습니다. 99.6309963099631%변동
KRW-META 42.8원
KRW-META은 10%이상 급등하지 않았습니다. 

KeyboardInterrupt: 